#**📚 📝 자기소개서 생성기 📊 🔍**
---
*KUBIG NLP 생성 2팀* _ 기다연, 전지우, 이현규

###**✅ 준비 과정으로써 아래 코드를 모두 돌려주세요 !**
  
⬇️ 괄호 사이의 재생 버튼을 눌러주세요 !

In [ ]:
!pip install transformers
import os
import pandas as pd
import numpy as np
import random
import re

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, BertTokenizerFast
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import time 
!if [ -e "/content/model.zip" ]; then rm "/content/model.zip"; fi # model.zip 가져오는 경우
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1bXhFT-tpj-8G-I3D2N2O6jJGjHo2GpNd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1bXhFT-tpj-8G-I3D2N2O6jJGjHo2GpNd" -O model.zip && rm -rf /tmp/cookies.txt
time.sleep(4)
!if [ -d "/content/model_save" ]; then rm -r "/content/model_save"; fi # model_save 가져오는 경우
!unzip /content/model.zip

--2021-09-02 11:45:00--  https://docs.google.com/uc?export=download&confirm=Q7jm&id=1bXhFT-tpj-8G-I3D2N2O6jJGjHo2GpNd
Resolving docs.google.com (docs.google.com)... 64.233.189.138, 64.233.189.101, 64.233.189.100, ...
Connecting to docs.google.com (docs.google.com)|64.233.189.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-ac-docs.googleusercontent.com/docs/securesc/ji41t8q9rkee2ft9hqt83lpldjokvfsv/ctt0r067glot42d51f65uq1gufn0cdhs/1630583100000/15097711384147757777/12537631320545482420Z/1bXhFT-tpj-8G-I3D2N2O6jJGjHo2GpNd?e=download [following]
--2021-09-02 11:45:00--  https://doc-14-ac-docs.googleusercontent.com/docs/securesc/ji41t8q9rkee2ft9hqt83lpldjokvfsv/ctt0r067glot42d51f65uq1gufn0cdhs/1630583100000/15097711384147757777/12537631320545482420Z/1bXhFT-tpj-8G-I3D2N2O6jJGjHo2GpNd?e=download
Resolving doc-14-ac-docs.googleusercontent.com (doc-14-ac-docs.googleusercontent.com)... 64.233.189.132, 2404:6800:4008:c07::84
Connecting

In [ ]:
# GPT2 모델, 토크나이저 불러오기
LOAD_DIR = '/content/model_save/model_batch1'
model = GPT2LMHeadModel.from_pretrained(LOAD_DIR)
tokenizer = BertTokenizerFast.from_pretrained(LOAD_DIR)
input_ids = tokenizer.encode("text to tokenize")[1:]  # CLS token 제거
tokenizer_gpt3 = BertTokenizerFast.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2", 
                                                   bos_token='<|startoftext|>', 
                                                   eos_token='<|endoftext|>', 
                                                   pad_token='<|pad|>')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


###**🖊 자기소개서를 생성해봅시다 !**
*데이터 양이 많아 시간이 조금 걸리는 점 양해 부탁드립니다.*


In [ ]:
#@markdown ⬅️  아래 항목들을 채우고 클릭해주세요 !

#@markdown 원하는 자기소개서의 첫 문장을 입력해주세요:

sentence = "\uC800\uB294 \uC5F4\uC815\uC801\uC778 \uC0AC\uB78C\uC785\uB2C8\uB2E4." #@param {type:"string"}

#@markdown 생성할 자기소개서 답변의 개수를 고르세요:
num = 3 #@param {type:"slider", min:1, max:10, step:1}

model.eval()

# input 생성하기
prompt = f"<|startoftext|> {sentence}"
generated = torch.tensor(tokenizer_gpt3.encode(prompt)[1:]).unsqueeze(0)
generated = generated.to(device)

# output 만들기
sample_outputs = model.generate(
                                  generated, 
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 500,
                                  top_p=0.95, 
                                  num_return_sequences=num,
                                  repetition_penalty=1.1
                                  )

print(" 📄 입력하신 '{}'을 기반으로 생성된 자기소개서입니다 !".format(sentence))

# 생성된 자기소개서 print
for i, sample_output in enumerate(sample_outputs):
    result = "{}\n\n".format(tokenizer_gpt3.decode(sample_output, skip_special_tokens=True))
    print(" {}번째 자기소개서: ".format(i+1), '\n', result)

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


 📄 입력하신 '저는 열정적인 사람입니다.'을 기반으로 생성된 자기소개서입니다 !
 1번째 자기소개서:  
 저는 열정적인 사람입니다. 대학 시절 공부했던 화학과목에서 조장을 맡아서 수행한 적이 있습니다. 학기마다 발표하는 조가 있었는데 매주 조원 명이 다 같이 모여 조에 대해 토론하고 발표를 준비해야 했습니다. 각자 한 조를 만들어야 했기 때문에 각 조의 담당 교수님과 한 시간씩 만나서 대화하는 시간을 가졌습니다. 그 날 주제가 정해졌는데 주제를 정하기가 쉽지 않았습니다. 가장 유력한 주제는 교수님의 승인이 필요한 주제였기에 교수님께 의견을 여쭤보니 이미 정해놓은 주제라 다른 주제로 진행하자고 하셨습니다. 하지만 기존의 주제가 싫지 않았고 자신의 전공과 맞지 않으면 새로운 주제를 선택해야 한다고 생각하여 발표방식을 바꾸어보기로 결정했습니다. 기존에는 발표연습을 하는 시간을 갖는 것이 전부였지만 제가 발표한 주제에 관한 지식들을 학생들에게 설명하는 시간을 가질 수도 있고 모르는 내용도 쉽게 물어보면서 발표를 진행할 수 있었습니다. 발표 후에는 교수님께서 이 정도면 되겠네라고 말씀해 주심에 더 열심히 하게 되었습니다. 하지만 이러한 노력에도 불구하고 발표내용이 너무 부족해 학생들로부터 좋은 평을 받지 못했습니다. 그래서 교수님에게 다시 질문했고 그 답은 저희의 연구결과를 어떻게 해석할 것인지에 대한 교수님의 생각이었고 그렇게 해석된 논문으로 실험을 하기로 결정했습니다. 실험을 진행하면서 기존 논문과 달리 결과가 빠르게 나올 뿐만 아니라 여러 분석기기들의 도움을 받으면서 실험을 진행한다면 더 빠르고 확실한 결과를 얻을 수 있다고 판단되어 연구실에 들어가 교수님을 대신해서 실험한 결과도 역시 빠르게 나와 주었습니다. 이러한 경험을 통해서 어떤 일을 진행할 때 어떤 방식으로 그 과정을 이끌어 나가야 할 지 빠르게 알아내고 그 과정에서 빠르게 개선점을 찾아 낼 수 있어야 한다는 것을 배울 수 있었습니다. 제가 입사하게 된다면 많은 업무 중에서 특히 중요하다고 생각되는 